In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

import csv
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

HEADERS = ['title', 'latitude', 'longitude']  
MARKUP_FILENAME = 'html-markup.txt'
CSV_DATA_FILENAME=  r'C:\Users\Professional\Downloads\data69.csv'

URL = f'https://2gis.kz/almaty/search/остановки

with open(CSV_DATA_FILENAME, 'w', encoding='utf-8-sig', newline='') as f:
    csv_writer = csv.DictWriter(f, fieldnames=HEADERS)
    csv_writer.writeheader()

def extract_url_from_element(element):
    # Пытаемся найти тег <a> со ссылкой внутри текущего элемента
    link = element.find('a', href=True)
    if link and 'href' in link.attrs:
        return link['href']
    return None

def extract_coordinates(script_text):
    match = re.search(r'centroid":"POINT\(([\d.]+) ([\d.]+)\)"', script_text)
    if match:
        longitude, latitude = match.groups()
        return latitude, longitude
    return None, None


def data_handler(page_number):
    with open(MARKUP_FILENAME, 'r', encoding='utf-8-sig') as f:
        contents = f.read()
    doc = BeautifulSoup(contents, features='html.parser')  
    parent_div_results = doc.find_all('div', class_='_1kf6gff')

    TITLES = []
    LATITUDES = []
    LONGITUDES = []
    URLS = [] 


    for idx, res_div in enumerate(parent_div_results):
        title = res_div.find('span', class_='_15a9jdw').text if res_div.find('span', class_='_15a9jdw') else 'null'
        TITLES.append(title)

        stop_url = extract_url_from_element(res_div)
        URLS.append(stop_url)

        if stop_url:
            browser.get('https://2gis.kz/'+ stop_url)
            WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "script")))
            scripts = browser.find_elements(By.TAG_NAME, 'script')
            found = False
            for script in scripts:
                if 'centroid' in script.get_attribute('innerHTML'):
                    latitude, longitude = extract_coordinates(script.get_attribute('innerHTML'))
                    if latitude and longitude:
                        LATITUDES.append(latitude)
                        LONGITUDES.append(longitude)
                        found = True
                        break

            if not found:
                LATITUDES.append('null')
                LONGITUDES.append('null')

            browser.back()
        else:
            LATITUDES.append('null')
            LONGITUDES.append('null')

               
    with open(CSV_DATA_FILENAME, 'a', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)

        for m in range(1, len(TITLES)):
            writer.writerow([TITLES[m], LATITUDES[m], LONGITUDES[m]])

    print(f'\nfinished parsing page: {page_number}') 



chrome_binary_path = 'C:/Program Files/Google/Chrome/Application/chrome.exe'
chromedriver_path = r'‪C:\Users\Professional\Downloads\chrome-win64.zip'

options = Options()
options.binary_location = 'C:/Program Files/Google/Chrome/Application/chrome.exe' 
browser = webdriver.Chrome(executable_path='C:/Users/Professional/Desktop/chromedriver.exe', chrome_options=options)

browser.maximize_window()
browser.get(URL)
browser.implicitly_wait(40)

page_element = browser.find_element(
    By.XPATH, "(//span[@class='_1xhlznaa'])[1]")

num_of_pages = (int(page_element.text)//12)+3 

try:
    try:
        scroll_container = browser.find_element(
            By.XPATH, "(//div[@class='_15gu4wr'])[3]")
    except NoSuchElementException:
        scroll_container = browser.find_element(
            By.XPATH, "(//div[@class='_15gu4wr'])[2]")
    finally:
        
        for page in range(1, num_of_pages):
            with open(MARKUP_FILENAME, 'w', encoding='utf-8') as f:
                f.write(browser.page_source)
       
            data_handler(page)
            WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='_5ocwns']//div[2]"))
            )

            scroll_container = browser.find_element(
                By.XPATH, "(//div[@class='_15gu4wr'])[3]"
            )

            browser.execute_script("arguments[0].scrollIntoView(false);", scroll_container)

            WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='_5ocwns']//div[2]"))
            )

            print(f"Обработка страницы {page} завершена. Переход на следующую страницу.")

            next_page_button = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//div[@class='_5ocwns']//div[2]"))
            )
            next_page_button.click()
            print(f"Перешел на страницу {page + 1}")

        
        

except IndexError:  
    print(f'Total pages parsed {page-1}')
    time.sleep(3)
finally:
    time.sleep(2)
    browser.quit()  